In [ ]:
from kazoo.client import KazooClient
from threading import Thread
import time

In [ ]:
import logging

from time import time, sleep
from multiprocessing import Process
logging.basicConfig()

from kazoo.client import KazooClient


class Philosopher(Thread):
    def __init__(self, root: str, _id: int, fork_path: str, duration: int = 100):
        super().__init__()
        self.url = f'{root}/{_id}'
        self.root = root
        self.fork = fork_path
        self.id = _id
        self.left_fork_id = _id
        self.right_fork_id = _id + 1 if _id + 1 < 5 else 0
        self.duration = duration

    def run(self):
        zk = KazooClient()
        zk.start()

        table_lock = zk.Lock(f'{self.root}', self.id)
        left_fork = zk.Lock(f'{self.root}/{self.fork}/{self.left_fork_id}', self.id)
        right_fork = zk.Lock(f'{self.root}/{self.fork}/{self.right_fork_id}', self.id)

        start = time()
        while time() - start < self.duration:
            print(f'Philosopher {self.id}', 'Im thinking')
            with table_lock:
                if len(left_fork.contenders()) == 0 and len(right_fork.contenders()) == 0:
                    left_fork.acquire()
                    right_fork.acquire()
                    
            if left_fork.is_acquired:
                print(f'Philosopher {self.id}', 'Im eating')
                sleep(1)
                left_fork.release()
                right_fork.release()
            sleep(0.5)
            
        zk.stop()
        zk.close()

In [ ]:
master_zk = KazooClient()
master_zk.start()

if master_zk.exists('/table'):
    master_zk.delete('/table', recursive=True)

master_zk.create('/table')
master_zk.create('/table/forks')
master_zk.create('/table/forks/1')
master_zk.create('/table/forks/2')
master_zk.create('/table/forks/3')
master_zk.create('/table/forks/4')
master_zk.create('/table/forks/5')

root = '/table'
fork_path = 'forks'
for i in range(5):
    p = Philosopher(root, i, fork_path, 20)
    p.start()

In [ ]:
master_zk.delete('/table/forks/1', recursive=True)